<a href="https://colab.research.google.com/github/leticiabac/TCC-BI_MASTER/blob/main/PROJ_busca_precos_medicamentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importação de bibliotecas utilizadas

In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
#from googlesearch import search
from selenium import webdriver
from datetime import datetime
import pandas as pd
import time

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

# Lista de Sites Consultados
A seguir é criada a lista de sites de farmácias onde os preços serão consultados.

Previamente, a estrutura das páginas html de cada site foi analisada e foi mapeada uma expressão que retorna o valor do preço apresentado na página. Para coleta das informações das páginas será utilizado o framewrok Selenium Webdriver, originalmente criado para realização de testes simulando o comportamento de aplicações em navegadores. Posteriormente será simulada a abertura do site da farmácia em um browser para que os dados de interesse sejam extraídos da página html gerada.

In [2]:
# Cria dicionario de Expressoes de Busca por site ("site":"expressao de busca de preço na página")
dict_expressoes_preco = {}


# Drogaria Venancio
site = "www.drogariavenancio.com.br"
expressao = "wd.find_element_by_xpath('//*[@id=\"price-component\"]/div/div[1]/div/div/div/div/div/p/strong').text[3:]"
dict_expressoes_preco[site] = expressao

# Drogaraia
site = "www.drogaraia.com.br"
expressao = "wd.find_element_by_xpath('/html/body/div[13]/div/div[4]/div/div[4]/div[1]/div[2]/div/form/div[2]/div[2]/div/div[3]/div/div/label/div/div/span/p[2]/span/span[2]').text"
dict_expressoes_preco[site] = expressao

# Pague Menos
site = "www.paguemenos.com.br"
expressao = "wd.find_element_by_xpath('/html/body/div[2]/div/div[1]/div/div/div/div[3]/div/div[2]/div[1]/section/div/div[3]/div/div[2]/div/div/div[1]/div/div[2]/div/div[2]/span/span/span[3]').text + ',' + wd.find_element_by_xpath('/html/body/div[2]/div/div[1]/div/div/div/div[3]/div/div[2]/div[1]/section/div/div[3]/div/div[2]/div/div/div[1]/div/div[2]/div/div[2]/span/span/span[5]').text"
dict_expressoes_preco[site] = expressao

# Drogaria Pacheco
site = "www.drogariaspacheco.com.br"
expressao = "wd.find_element_by_xpath('//strong[@class=\"skuBestPrice\"]').text[3:]"
dict_expressoes_preco[site] = expressao

# Farmadelivery
site = "www.farmadelivery.com.br"
expressao = "wd.find_element_by_xpath('//p[2][@class=\"special-price\"]/span[2]').text[2:]"
dict_expressoes_preco[site] = expressao


# Busca URLs

A busca por URLs foi segregada para outro notebook para evitar excesso de acessos ao google. 

Aqui está sendo importado um arquivo .csv que já contenha as URLs levantadas para cada remedio.

In [3]:
df_urls = pd.read_csv('/content/lista_URLs_2021-10-24.csv',usecols=[1,2])  



In [4]:
print(df_urls.to_string())


                                  Remedio                                                                                                               Url
0           Puran T4 75mcg 30 comprimidos                                https://www.drogariavenancio.com.br/Puran-T4-75mcg-Sanofi-Aventis-30-Comprimidos/p
1           Puran T4 75mcg 30 comprimidos                                         https://www.drogaraia.com.br/puran-t4-75-mcg-uso-oral-30-comprimidos.html
2           Puran T4 75mcg 30 comprimidos                                                 https://www.paguemenos.com.br/puran-t4-75mcg-com-30-comprimidos/p
3           Puran T4 75mcg 30 comprimidos                                               https://www.drogariaspacheco.com.br/puran-t4-75mcg-30-comprimidos/p
4           Puran T4 75mcg 30 comprimidos                                                  https://www.farmadelivery.com.br/puran-t4-75mcg-c-30-comprimidos
5          Puran T4 88 mcg 30 comprimidos                       

#Cria e configura webdriver

In [5]:
#cria e configura webdriver
wait_imp = 20
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('ignore-certificate-errors')
wd = webdriver.Chrome('chromedriver',options=chrome_options)

# Coleta de informações

Percorre cada um dos sites de farmácias mapeados extraindo a informação do preço atual disponível no site.

In [6]:
#Inicia array de precos
precos = []

#Percorre URLs coletando os precos
for index, row in df_urls.iterrows():
  remedio = row["Remedio"]
  url = row["Url"]
  print(url)
  if(url == "ND"):
    continue
  wd.get(url)
  wd.implicitly_wait(wait_imp)
  site = url.split('/')[2]
  try:
    preco = eval(dict_expressoes_preco.get(site))
  except:
    preco = "ND"
  precos.append([remedio,preco,site])

print(precos)


https://www.drogariavenancio.com.br/Puran-T4-75mcg-Sanofi-Aventis-30-Comprimidos/p


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.


https://www.drogaraia.com.br/puran-t4-75-mcg-uso-oral-30-comprimidos.html
https://www.paguemenos.com.br/puran-t4-75mcg-com-30-comprimidos/p
https://www.drogariaspacheco.com.br/puran-t4-75mcg-30-comprimidos/p
https://www.farmadelivery.com.br/puran-t4-75mcg-c-30-comprimidos
https://www.drogariavenancio.com.br/puran-t4-88mcg-sanofi-aventis-30-comprimidos/p
https://www.drogaraia.com.br/puran-t4-88mcg-30-comprimidos.html
ND
https://www.drogariaspacheco.com.br/puran-t4-88mcg-sanofi-aventis-30-comprimidos/p
https://www.farmadelivery.com.br/puran-t4-88mcg-c-30-comprimidos
https://www.drogariavenancio.com.br/Addera-D3-7-000ui-Hypera-4-Comprimidos-Revestidos/p
https://www.drogaraia.com.br/addera-d3-7000-ui-1-blister-com-4-comprimidos.html
https://www.paguemenos.com.br/addera-d3-7000ui-com-4-comprimidos/p
https://www.drogariaspacheco.com.br/addera-d3-7000ui-mantecorp-farmasa-4-comprimidos/p
https://www.farmadelivery.com.br/addera-d3-7000ui-c-4-comprimidos
https://www.drogariavenancio.com.br/Adder

Cria Pandas Dataframe

In [7]:
df_precos = pd.DataFrame(precos, columns =['Remedio','Preco', 'Fonte']) 
print(df_precos)

                                  Remedio   Preco                        Fonte
0           Puran T4 75mcg 30 comprimidos   14,89  www.drogariavenancio.com.br
1           Puran T4 75mcg 30 comprimidos   15,06         www.drogaraia.com.br
2           Puran T4 75mcg 30 comprimidos   14,65        www.paguemenos.com.br
3           Puran T4 75mcg 30 comprimidos   14,80  www.drogariaspacheco.com.br
4           Puran T4 75mcg 30 comprimidos   13,44     www.farmadelivery.com.br
5          Puran T4 88 mcg 30 comprimidos   16,79  www.drogariavenancio.com.br
6          Puran T4 88 mcg 30 comprimidos   16,64         www.drogaraia.com.br
7          Puran T4 88 mcg 30 comprimidos   16,47  www.drogariaspacheco.com.br
8          Puran T4 88 mcg 30 comprimidos   16,19     www.farmadelivery.com.br
9         Addera d3 7000 UI 4 comprimidos   34,90  www.drogariavenancio.com.br
10        Addera d3 7000 UI 4 comprimidos   30,19         www.drogaraia.com.br
11        Addera d3 7000 UI 4 comprimidos   23,75   

Transforma os preços em colunas por Farmácia

In [8]:
df_precos = df_precos.groupby(['Remedio','Fonte'])['Preco'].first().unstack()

print(df_precos)

Fonte                                 www.drogaraia.com.br  ... www.paguemenos.com.br
Remedio                                                     ...                      
Addera d3 7000 UI 10 comprimidos                     61,99  ...                 50,45
Addera d3 7000 UI 30 comprimidos                     96,88  ...                 96,25
Addera d3 7000 UI 4 comprimidos                      30,19  ...                 23,75
Ganfort 3ml                                         116,99  ...                117,85
Ganfort 5ml                                         191,99  ...                191,55
Livalo 2mg 30 comprimidos                            95,49  ...                 92,65
Milgamma 150mg 30 comprimidos                        59,99  ...                 56,15
Nesina pio 25mg + 30mg 30 comprimidos                   ND  ...                141,75
Puran T4 75mcg 30 comprimidos                        15,06  ...                 14,65
Puran T4 88 mcg 30 comprimidos                       1

Salva em um arquivo (.csv)

In [9]:
data = datetime.now().strftime("%Y-%m-%d")
df_precos.to_csv('lista_Precos_' + data + '.csv')